## NAME ENTITY RECOGNITION APPLICATION

### The following code uses the `dslim/bert-base-NER` model to identify PERson , LOCation and an ORGanization

In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = os.environ['HF_API_KEY']

In [2]:
# Helper function
import requests, json
import gradio as gr
#Summarization endpoint
def get_completion(inputs, parameters=None, ENDPOINT_URL=os.environ['HF_API_NER_BASE']): 
    headers = {
      "Authorization": f"Bearer {hf_api_key}",
      "Content-Type": "application/json"
    }
    data = { "inputs": inputs }
    if parameters is not None:
        data.update({"parameters": parameters})
    response = requests.request("POST",ENDPOINT_URL, headers=headers,data=json.dumps(data))
    return json.loads(response.content.decode("utf-8"))

#### The merge function is used here to merge the tokens that are correctly identified to make them one.

In [3]:
HF_API_NER_BASE = os.environ['HF_API_NER_BASE'] #NER endpoint

def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['entity_group'].startswith('PER') and merged_tokens[-1]['entity_group'].endswith(token['entity_group'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens



In [4]:
def ner(input):
    output = get_completion(input, parameters=None)
    merged_tokens = merge_tokens(output)
    return {"text": input, "entities": merged_tokens}

gr.close_all()

demo = gr.Interface(fn=ner,
                    inputs=[gr.Textbox(label="Text to find entities", lines=5)],
                    outputs=[gr.HighlightedText(label="Text with entities")],
                    title="NER with dslim/bert-base-NER",
                    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
                    allow_flagging="never",
                    #Here we introduce a new tag, examples, easy to use examples for your application
                    examples=["My name is Atique and I live in Peshawar",
                              "My friends name is Abrar, he is from Swabi but lives in Islamabad"])
demo.launch(share = True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e32bcb65ce17323b22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
gr.close_all()